# Lesson 5: Creating an MCP Client 

In the previous lesson, you created an MCP research server that exposes 2 tools. In this lesson, you will make the chatbot communicate to the server through an MCP client. This will make the chatbot MCP compatible. You will continue from where you left off in lesson 4, i.e., you are provided again with the `mcp_project` folder that contains the `research_server.py` file. You'll add to it the MCP chatbot file and update the environment. 

<img src="images/lesson_progression.png" width="700">

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the  <code>mcp_project</code> folder :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>L5</em>.
</div>

## Back to the Chatbot Example

Here are the main code parts (`process_query`, `chat_loop`) from the chatbot example of lesson 3. Notice that the burden of tool definitions and execution is now shifted onto the MCP server, so the chatbot logic only contains code related to processing the user queries and to keeping the chat loop running until the user types `quit`.

In [10]:
import ollama 
from ollama import Client

client = Client()

response = client.chat(model='qwen3:1.7b', messages=[
  {
    'role': 'user',
    'content': 'what is the capital city of Japan',
  },
])

In [11]:
response

ChatResponse(model='qwen3:1.7b', created_at='2025-06-17T19:17:10.886973Z', done=True, done_reason='stop', total_duration=18468317300, load_duration=13989376600, prompt_eval_count=15, prompt_eval_duration=709344900, eval_count=178, eval_duration=3768019400, message=Message(role='assistant', content="<think>\nOkay, the user is asking for the capital city of Japan. Let me think. I remember that Japan is a country in East Asia, and their capital is Tokyo. But I should verify that. Wait, is there any chance that the capital could be another city? Like maybe Osaka or Kyoto? No, I think Tokyo is the main capital. Let me recall: the Japanese government is based in Tokyo, and it's the largest city in Japan. Yes, I'm pretty sure. So the answer should be Tokyo. I should make sure there's no confusion with other cities. For example, some countries have multiple capitals, but Japan's capital is definitely Tokyo. Alright, I think that's solid.\n</think>\n\nThe capital city of Japan is **Tokyo**. It 

In [1]:
from ollama import chat

chat_response = chat(
    model='gemma3:1b',
    messages=[{'role': 'user', 'content': 'what is the capital city of Russia'}],
)

In [2]:
chat_response

ChatResponse(model='gemma3:1b', created_at='2025-06-17T19:20:33.1463469Z', done=True, done_reason='stop', total_duration=6690740100, load_duration=4557004800, prompt_eval_count=16, prompt_eval_duration=666777300, eval_count=37, eval_duration=1462118200, message=Message(role='assistant', content='The capital city of Russia is **Moscow**. \n\nWhile Saint Petersburg is the largest city in Russia, Moscow is the political, economic, and cultural center of the country. 😊', images=None, tool_calls=None))

In [13]:
key =  chat_response.message.tool_calls
key = "some"
not key

False

In [38]:
message = chat_response.message.content
message = message[message.index('</think>')+8:]
print(message.strip())

The capital city of Russia is **Moscow**. It is the largest city in Russia and the political, economic, and cultural center of the country.


In [3]:
#from dotenv import load_dotenv
from ollama import chat

#load_dotenv()

def process_query(query):
    messages = [{'role':'user', 'content':query}]
    response = chat(
        model='gemma3:1b',
        messages=messages
    )
    process_query = True
    while process_query:
        if not response.message.tool_calls:
            # If there are no tool calls, just print the response and exit
            reply = response.message.content
            if '</think>' in reply:
                reply = reply[reply.index('</think>')+8:]
            process_query = False
            print(reply.strip())
        else:
            # Handle tool calls
            for tool_call in response.message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = tool_call.function.arguments
                tool_id = tool_call.id

                print(f"Calling tool {tool_name} with args {tool_args}")
                
                # Execute the tool
                result = execute_tool(tool_name, tool_args)
                
                # Add the tool result to messages
                messages.append({
                    "role": "assistant",
                    "content": response.message.content,
                    "tool_calls": [tool_call]
                })
                messages.append({
                    "role": "user",
                    "content": str(result),
                    "tool_call_id": tool_id
                })

                # Get new response with tool result
                response = chat(
                    model='qwen3:1.7b',
                    messages=messages
                )

                if not response.message.tool_calls:
                    reply = response.message.content
                    if '</think>' in reply:
                        reply = reply[reply.index('</think>')+8:]
                    print(reply.strip())
                    process_query = False
    print(messages)

def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [4]:
chat_loop()

Type your queries or 'quit' to exit.
1 + 1 = 2

It’s a fundamental concept in math! 😊 

Let me know if you’d like to learn more about it!
[{'role': 'user', 'content': 'hello what is 1+1?'}]


Okay, no problem at all! 😊 Is there anything else I can help you with?
[{'role': 'user', 'content': 'nope thanks'}]




## Building your MCP Client

Now you will take the functions `process_query` and `chat_loop` and wrap them in a `MCP_ChatBot` class. To enable the chatbot to communicate to the server, you will add a method that connects to the server through an MCP client, which follows the structure given in this reference code:

### Reference Code
``` python
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="uv",  # Executable
    args=["run example_server.py"],  # Command line arguments
    env=None,  # Optional environment variables
)

async def run():
    # Launch the server as a subprocess & returns the read and write streams
    # read: the stream that the client will use to read msgs from the server
    # write: the stream that client will use to write msgs to the server
    async with stdio_client(server_params) as (read, write): 
        # the client session is used to initiate the connection 
        # and send requests to server 
        async with ClientSession(read, write) as session:
            # Initialize the connection (1:1 connection with the server)
            await session.initialize()

            # List available tools
            tools = await session.list_tools()

            # will call the chat_loop here
            # ....
            
            # Call a tool: this will be in the process_query method
            result = await session.call_tool("tool-name", arguments={"arg1": "value"})


if __name__ == "__main__":
    asyncio.run(run())
`````

- `async with stdio_client(server_params) as (read, write): `  defines a context manager to first pass in parameters from our servers and establish a connection as subprocess using the `async`. Once we establishes a server connection we're going to get access to a read and write stream that we can then pass to a heigher level class called the `ClientSession`.

- In this Client Session when we pass the read and write stream we get access to an underlying connection that allow us to make use of functionality for listing tools, itilializing connections, and doing quite a bit more with other primitive. 

- The client's job is to query for available tools and take those tool and pass them to a LLM. If there is a need to invoke the MCP server will invoke it and if a tool needs to be executed we'll let the MCP server know what to do. 

- Since we're using an async enviromnent we'll be moving past the MCP.run and will be using asyncio.run 

### Adding MCP Client to the Chatbot

The MCP_ChatBot class consists of the methods:
- `process_query`
- `chat_loop`
- `connect_to_server_and_run`
  
and has the following attributes:
- session (of type ClientSession)
- anthropic: Anthropic                           
- available_tools

In `connect_to_server_and_run`, the client launches the server and requests the list of tools that the server provides (through the client session). The tool definitions are stored in the variable `available_tools` and are passed in to the LLM in `process_query`.

<img src="images/tools_discovery.png" width="400">


In `process_query`, when the LLM decides it requires a tool to be executed, the client session sends to the server the tool call request. The returned response is passed in to the LLM. 

<img src="images/tool_invocation.png" width="400">

Here're the `mcp_chatbot` code.

In [ ]:
%%writefile mcp_project/mcp_chatbot.py
from dotenv import load_dotenv
from ollama import chat
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List
import asyncio
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.session: ClientSession = None
        self.available_tools: List[dict] = []

    async def process_query(self, query):
        messages = [{'role':'user', 'content':query}]
        response = chat(
            model='qwen3:1.7b',
            messages=messages
        )
        process_query = True
        while process_query:
            if not response.message.tool_calls:
                # If there are no tool calls, just print the response and exit
                print(response.message.content)
                process_query = False
            else:
                # Handle tool calls
                for tool_call in response.message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = tool_call.function.arguments
                    tool_id = tool_call.id

                    print(f"Calling tool {tool_name} with args {tool_args}")
                    
                    # Execute the tool
                    #result = execute_tool(tool_name, tool_args): not anymore needed
                    # tool invocation through the client session
                    result = await self.session.call_tool(tool_name, arguments=tool_args)
                    messages.append({
                        "role": "assistant",
                        "content": response.message.content,
                        "tool_calls": [tool_call]
                    })
                    messages.append({
                        "role": "user",
                        "content": str(result),
                        "tool_call_id": tool_id
                    })

                    # Get new response with tool result
                    response = chat(
                        model='qwen3:1.7b',
                        messages=messages
                    )

                    if not response.message.tool_calls:
                        print(response.message.content)
                        process_query = False

    
    
    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        
        while True:
            try:
                query = input("\nQuery: ").strip()
        
                if query.lower() == 'quit':
                    break
                    
                await self.process_query(query)
                print("\n")
                    
            except Exception as e:
                print(f"\nError: {str(e)}")
    
    async def connect_to_server_and_run(self):
        # Create server parameters for stdio connection
        server_params = StdioServerParameters(
            command="uv",  # Executable
            args=["run", "research_server.py"],  # Optional command line arguments
            env=None,  # Optional environment variables
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                # Initialize the connection with underlying stuff
                await session.initialize()
    
                # List available tools
                response = await session.list_tools()
                
                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])
                
                self.available_tools = [{
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                } for tool in response.tools]
    
                await self.chat_loop()


async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()
  

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/mcp_chatbot.py


## Running the MCP Chatbot

**Terminal Instructions**

- To open the terminal, run the cell below.
- Navigate to the `mcp_project` directory:
    - `cd L5/mcp_project`
- Activate the virtual environment:
    - `source .venv/bin/activate`
- Install the additional dependencies:
    - `uv add anthropic python-dotenv nest_asyncio`
- Run the chatbot:
    - `uv run mcp_chatbot.py`
- To exit the chatbot, type `quit`.
- If you run some queries and would like to access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L5` -> `mcp_project`.

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Resources

- [Quick Start for Client Developpers](https://modelcontextprotocol.io/quickstart/client)
- [Writing MCP client](https://github.com/modelcontextprotocol/python-sdk/blob/main/examples/clients/simple-chatbot/mcp_simple_chatbot/main.py)
- [Another mcp chatbot example](https://github.com/modelcontextprotocol/python-sdk/blob/main/examples/clients/simple-chatbot/mcp_simple_chatbot/main.py)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the  <code>mcp_project</code> folder :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and then 3) on "L5".
<p> ⬇ &nbsp; <b>To Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>